In [1]:
import numpy as np
import cv2
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import scipy.interpolate
import astropy.units as u
import astropy

import simdata

%reload_ext autoreload
%autoreload 2
%load_ext line_profiler
from importlib import reload
from pprint import pprint
from src.vortector.vortector import Vortector
from src.vortector.visualize import show_fit_overview_2D

In [2]:
# very hard example overlapping with spiral arm

# simid = "4ae2169c"
# Noutput = 30

# hard example

# simid = "306d9f0c"
# Noutput = 52

# overlapping with boundary example

# simid = "9f976424"
# Noutput = 90

# spiral arm artifact

# simid = "487fa644"
# Noutput = 223

# big vortex example
simid = "3625e016"
Noutput = 200

# easy example

# simid = "a122b63c"
# Noutput = 75

# # faint vortex
# simid = "65a701a4"
# Noutput = 380

# another faint vortex
# simid = "b522b88a"
# Noutput = 116

# no vortex

# simid = "3401c4d0"
# Noutput = 256

# 8cps x 2cps example

# simid = "65a701a4"
# Noutput = 50

# 16 cps example
# simid = "20ce240c"
# Noutput = 82 # overlapping with spiral, need 0.2 spacing in vortensity
# Noutput = 83
# Noutput = 84 # hard case barely passing

# easier 16 cps example
# simid = "71cfb245"
# Noutput = 54

# transition disk example
# vortex in migration jump paper
# simid = "d132cf47"
# Noutput = 524

# multiple vortices example
# simid = "b522b88a"
# Noutput  = 10


## weak vortex inspection
# simid = "5071c355"
# Noutput = 146

simulation = simdata.SData(simid)

levels = [float(x) for x in np.arange(-1,1.5,0.05)]

# if "run_vortector" in locals() and "calc_quantities" in locals():
#     print("running vortector")
#     X, Y, Xc, Yc, A, vortensity, vorticity, Rho, Rho_background = calc_quantities()
#     run_vortector()

## Vorticity calculation for simdata

In [3]:
from simdata_vorticity import vorticity_simdata, map_angles, calc_quantities
X, Y, Xc, Yc, A, vortensity, vorticity, Rho, Rho_background = calc_quantities(simulation, Noutput)

In [2]:
Rlims = [5.2, 12]
nl = np.argmin(np.abs(Xc[:,0]-Rlims[0]))
nr = np.argmin(np.abs(Xc[:,0]-Rlims[1]))
vd = Vortector(Xc[nl:nr,:], Yc[nl:nr,:], A[nl:nr,:], vortensity[nl:nr,:], Rho[nl:nr,:],
                         verbose=False, med=0.15, mear=np.inf,
                         levels=levels
                        )
# vd = vortector.Vortector(Xc, Yc, A, vortensity, Rho, Rho_background,
#                          [40,80], verbose=False, med=0.15, mear=np.inf,
#                          levels=levels
#                         )
%time vortices = vd.detect_vortices(include_mask=True, keep_internals=False)
for v in vd.vortices:
    try:
        v["strength"] = np.exp(-v["vortensity_median"])*v["mass"]
        print("strength = {:.2e}, mass = {:.2e} , min vort = {:.3f}, sigma fit r_diff = {:.2e} phi_diff = {:.2e}".format(
            v['strength'],v['mass'],v['vortensity_min'],v["sigma_fit_r_reldiff"],v["sigma_fit_phi_reldiff"]))
        print(f"    contour diff 2D = {v['sigma_fit_contour_diff_2D']:.2e} , contour reldiff 2D = {v['sigma_fit_contour_reldiff_2D']:.2e}")
        print(f"    ellipse diff 2D = {v['sigma_fit_ellipse_diff_2D']:.2e} , ellipse reldiff 2D = {v['sigma_fit_ellipse_reldiff_2D']:.2e}")
        print(f"    area_ratio_ellipse_to_contour = {v['sigma_fit_area_ratio_ellipse_to_contour']:.2e}")
    except KeyError:
        pass

# vd.show_fit_overview_1D(0)
show_fit_overview_2D(vd)
s = vd.vortices[0]["contour"]["stats"]
s["vortensity_min"]/s["azimuthal_at_vortensity_min"]["vortensity_med"]

NameError: name 'Xc' is not defined

In [ ]:
vd.vortices[0]

In [ ]:
vrad = simulation.fluids["gas"].get("2d", "velocity radial", Noutput).data.to_value("au/yr")
vrad = 0.5*(vrad[1:,:] + vrad[:-1,:])
vphi = simulation.fluids["gas"].get("2d", "velocity azimuthal", Noutput)
omega_frame = simulation.planets[0].get("omega frame").get_closest_to_time(vphi.get_time()).to("yr-1")
v_Frame = omega_frame*Xc*u.au
vphi = vphi.data.to_value("au/yr") + v_Frame.to_value("au/yr")
# vphi = vphi.data.to_value("au/yr")
vphi = np.pad(vphi, [[0,0],[0,1]], mode="wrap")
vphi = 0.5*(vphi[:,1:] + vphi[:,:-1])

vrad = vrad[nl:nr,:]
vphi = vphi[nl:nr,:]

2*np.pi*u.au/u.yr/np.sqrt(5.2)

In [ ]:
(astropy.constants.G*u.solMass).to_value("au3/yr2")

In [ ]:
astropy.constants.G.to("au3 yr-2 solMass-1")

In [ ]:
def orbital_elements(r, phi, area, rho, vrad, vphi, mu, mask=None):

    #mass of each cell
    mass = rho * area

    #velocities in cartesian for each cell
    vx = vrad*np.cos(phi) - vphi*np.sin(phi)
    vy = vrad*np.sin(phi) + vphi*np.cos(phi)
    
    #cartesian coordinates
    x  = r * np.cos(phi)
    y  = r * np.sin(phi)

    #specific angular momentum and speed squared
    h2  = (x*vy - y*vx)**2
    v2  = vx*vx + vy*vy

    #smj axis from energy
    eng = 0.5*v2 - mu/r
    a   = -0.5*mu/eng
    
    #eccentricity
    ecc = np.sqrt(1.0-h2/(mu*a))
    
    #weight by mass, calculate weighted eccentricity of each cell
    if mask is None:
        mask = np.ones(mass.shape, dtype=bool)
        
    mass = mass[mask]
    a = a[mask]
    ecc = ecc[mask]
    
    total_mass = np.sum(mass)
    
    weighted_a = np.sum(a*mass)/total_mass
    weighted_ecc = np.sum(ecc*mass)/total_mass

    return {"a" : weighted_a, "e" : weighted_ecc}
mu = (astropy.constants.G*u.solMass).to_value("au3/yr2")

In [ ]:
orbital_elements(vd.radius, vd.azimuth, vd.area, vd.surface_density, vrad, vphi, mu)

In [5]:
orbital_elements(vd.radius, vd.azimuth, vd.area, vd.surface_density, vrad, vphi, mu, mask=vd.vortices[0]["contour"]["mask"])

NameError: name 'orbital_elements' is not defined

In [6]:
r0 = vd.vortices[0]["fits"]["surface_density"]["r0"]
hr = np.sqrt(2*np.log(2))*vd.vortices[0]["fits"]["surface_density"]["sigma_r"]
phi0 = vd.vortices[0]["fits"]["surface_density"]["phi0"]
hphi = np.sqrt(2*np.log(2))*vd.vortices[0]["fits"]["surface_density"]["sigma_phi"]
ellipse_mask = ((vd.radius - r0)/hr)**2 + ((vd.azimuth - phi0)/hphi)**2 <= 1
orbital_elements(vd.radius, vd.azimuth, vd.area, vd.surface_density, vrad, vphi, mu, mask=ellipse_mask)

AttributeError: 'Vortector' object has no attribute 'vortices'

In [7]:
from src.vortector import analyze
analyze.calc_orbital_elements(vd.radius, vd.azimuth, vd.area, vd.surface_density, vrad, vphi, mu, mask=ellipse_mask)

NameError: name 'vrad' is not defined

In [14]:
analyze.calc_orbital_elements_vortector(vd, 0, vrad, vphi, mu, region="contour")

{'a': 8.208406271102724, 'e': 0.018638957260289496}

In [15]:
analyze.calc_orbital_elements_vortector(vd, 0, vrad, vphi, mu, region="vortensity")

{'a': 8.18388628152948, 'e': 0.019452842032688326}

In [16]:
analyze.calc_orbital_elements_vortector(vd, 0, vrad, vphi, mu, region="surface_density")

{'a': 8.200570204684256, 'e': 0.01700690411244635}